In [3]:
#packages used
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
import pandas as pd
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, TimeDistributed, Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy

In [4]:
#importing data as pandas data frame
DevData = pd.read_json("dev.jsonl", lines = True)
TestData = pd.read_json("test.jsonl", lines = True)
TrainData = pd.read_json("train.jsonl", lines = True)

In [5]:
TrainData

,premise,choice1,choice2,question,label,idx
0,My body cast a shadow over the grass.,The sun was rising.,The grass was cut.,cause,0,0
1,The woman tolerated her friend's difficult beh...,The woman knew her friend was going through a ...,The woman felt that her friend took advantage ...,cause,0,1
2,The women met for coffee.,The cafe reopened in a new location.,They wanted to catch up with each other.,cause,1,2
3,The runner wore shorts.,The forecast predicted high temperatures.,She planned to run along the beach.,cause,0,3
4,The guests of the party hid behind the couch.,It was a surprise party.,It was a birthday party.,cause,0,4
...,...,...,...,...,...,...
345,The photograph faded.,It was fake.,It was old.,cause,1,345
346,The boy got a black eye.,The bully mocked the boy.,The bully punched the boy.,cause,1,346
347,I misplaced my wallet.,I retrieved my phone.,I retraced my steps.,effect,1,347
348,The hunter ran out of ammunition.,He reloaded the gun.,He aimed at the deer.,effect,0,348


In [6]:
#tokenizes test data
TrainTok = Tokenizer(lower=True, split=' ',)

TrainTok.fit_on_texts(TrainData['choice2'])        
TrainTok.fit_on_texts(TrainData['choice1'])
TrainTok.fit_on_texts(TrainData['premise'])


In [41]:
TrainConfig = TrainTok.get_config()

#dictionary of words and indexs
TrainVocab = eval(TrainConfig['index_word'])

#finds length of vocab
TrainVocabSize = len(TrainVocab)

print("there are ", TrainVocabSize, " unique words")

there are  1740  unique words


In [78]:
PremiseTokTrain = TrainTok.texts_to_sequences(TrainData['premise'])
Choice1TokTrain = TrainTok.texts_to_sequences(TrainData['choice1'])
Choice2TokTrain = TrainTok.texts_to_sequences(TrainData['choice2'])

PremisePadTrain = sequence.pad_sequences(PremiseTokTrain, padding='post', maxlen=max(len(i) for i in PremiseTokTrain) + 2)
Choice1PadTrain = sequence.pad_sequences(PremiseTokTrain, padding='post', maxlen=max(len(i) for i in Choice1TokTrain) + 2)
Choice2PadTrain = sequence.pad_sequences(PremiseTokTrain, padding='post', maxlen=max(len(i) for i in Choice2TokTrain) + 2)

In [79]:
TrainCat = tf.concat([PremisePadTrain, Choice1PadTrain,Choice2PadTrain], 1)
 

In [103]:
def BILSTM(vocab_size, embedding_dim, rnn_units):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(rnn_units)),
    tf.keras.layers.Dense(1, activation='sigmoid')
  ])
  return model

In [104]:
model = BILSTM(
  vocab_size = TrainVocabSize,
  embedding_dim = 64,
  rnn_units = 64)
model.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_18 (Embedding)     (None, None, 64)          111360    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 128)               66048     
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 129       
Total params: 177,537
Trainable params: 177,537
Non-trainable params: 0
_________________________________________________________________


In [105]:
model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy', 'Precision', 'Recall'])

model.fit(TrainCat,TrainData["label"], epochs=100)

Epoch 1/100


2021-11-03 15:40:34.280436: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-11-03 15:40:34.474383: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-11-03 15:40:34.484773: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-11-03 15:40:34.664275: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-11-03 15:40:34.677665: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


11/11 [==============================] - 2s 65ms/step - loss: 0.6938 - accuracy: 0.4771 - precision: 0.4906 - recall: 0.7360
Epoch 2/100
11/11 [==============================] - 0s 37ms/step - loss: 0.6935 - accuracy: 0.4943 - precision: 0.5017 - recall: 0.8146
Epoch 3/100
11/11 [==============================] - 0s 40ms/step - loss: 0.6923 - accuracy: 0.5343 - precision: 0.5243 - recall: 0.9101
Epoch 4/100
11/11 [==============================] - 0s 35ms/step - loss: 0.6916 - accuracy: 0.5086 - precision: 0.5089 - recall: 0.9663
Epoch 5/100
11/11 [==============================] - 0s 33ms/step - loss: 0.6910 - accuracy: 0.5314 - precision: 0.5217 - recall: 0.9438
Epoch 6/100
11/11 [==============================] - 0s 33ms/step - loss: 0.6903 - accuracy: 0.5229 - precision: 0.5165 - recall: 0.9663
Epoch 7/100
11/11 [==============================] - 0s 33ms/step - loss: 0.6885 - accuracy: 0.5686 - precision: 0.5487 - recall: 0.8539
Epoch 8/100
11/11 [==============================] - 

In [107]:
PremiseTokTest = TrainTok.texts_to_sequences(TestData["premise"])
Choice1TokTest = TrainTok.texts_to_sequences(TestData["choice1"])
Choice2TokTest = TrainTok.texts_to_sequences(TestData["choice2"])

PremisePadTest = sequence.pad_sequences(PremiseTokTest, padding='post', maxlen=max(len(i) for i in PremiseTokTrain) + 2)
Choice1PadTest = sequence.pad_sequences(Choice1TokTest, padding='post', maxlen=max(len(i) for i in Choice1TokTrain) + 2)
Choice2PadTest = sequence.pad_sequences(Choice2TokTest, padding='post', maxlen=max(len(i) for i in Choice2TokTrain) + 2)


TestCat = tf.concat([PremisePadTest, Choice1PadTest,Choice2PadTest], 1)

model.evaluate(TestCat,TestData["label"])

4/4 [==============================] - 0s 20ms/step - loss: 3.5711 - accuracy: 0.4600 - precision: 0.4182 - recall: 0.5111


[3.5711185932159424, 0.4599999785423279, 0.418181836605072, 0.5111111402511597]